In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from datetime import datetime as dt
from PIL import Image
import time
import os
import re


In [36]:
path = r'./chromedriver'
driver = webdriver.Chrome(executable_path = path)
driver.get('https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/teams/poland/squad')

time.sleep(3)
SCROLL_PAUSE_TIME = 0.5
driver.implicitly_wait(3)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
i = 0
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    i+=1

#players1 = driver.find_elements(By.CLASS_NAME, 'player-badge-card_playerImage__301X0') 
players2 = driver.find_elements(By.CLASS_NAME, 'player-badge-card_badgeCard__2DJ4B') 

In [ ]:
for player in players2:
    index = [m.start() for m in re.finditer('\n', player.text)]
    file_name = player.text[:index[0]] +'_'+ player.text[index[0]+1:index[1]]
    test = player.find_elements(By.CLASS_NAME,"player-badge-card_playerImage__301X0")
    test = test[0].value_of_css_property("background-image")
    test = test[5:][:len(test[0])-3]

    img_content = requests.get(test).content
    img_file = io.BytesIO(img_content)

    image = Image.open(img_file)
    file_pth = r'/Users/marcinpestka/Desktop/python/'+file_name+'.png'

    with open(file_pth, 'wb') as file:
        image.save(file, 'png')
    i+=1

driver.close()
